In [9]:
from bidder import *
import websocket
import json
account = "MKS43148972"
venue = "CEX"
stock = "AVYO"
header['X-Starfighter-Authorization'] = "c8bbc206d27988125aa25e396f2f290f22109dbe"

import functools
import random

In [10]:
amount, position = 0, 0
avgasksize, avgbidsize = 0, 0

In [ ]:
amount = 

In [11]:
def check_anamoly(ordersize, avgsize):
    avgsize = (avgsize + ordersize + 0.00)/2
    if ordersize/avgsize < 0.1:
        return True, avgsize
    return False, avgsize

In [12]:
def quote_listener(account, ws, message):
    global amount, position, avgasksize, avgbidsize
    quote = json.loads(message).get('quote', None)
    if quote is None:
        return
    
    myrate = abs(amount/position) if not position == 0 else 0
    
    if position < 200:
        if 'ask' in quote:
            anamoly, avgasksize = check_anamoly(avgasksize, quote['ask'])
            if not anamoly and quote['ask'] < myrate - 400:
                size = quote['askSize'] if (position - quote['askSize']) < 200 else 200
                res = buy(account, venue, stock, quote['ask'], size, 'immediate-or-cancel')
                if res['ok']:
                    amount -= res['totalFilled'] * res['price']
                    position += res['totalFilled']
                
    if position > -200:
        if 'bid' in quote:
            anamoly, avgbidsize = check_anamoly(avgbidsize, quote['bid'])
            if not anamoly and quote['bid'] > myrate + 400:
                size = quote['bidSize'] if (position - quote['bidSize']) > -200 else 200
                res = sell(account, venue, stock, quote['bid'], size, 'immediate-or-cancel')
                if res['ok']:
                    amount += res['totalFilled'] * res['price']
                    position -= res['totalFilled']
    
    

In [13]:
quote_websocket(account, venue, stock, functools.partial(quote_listener, account))

<Thread(Thread-4, started daemon 140231621736192)>

<Thread(Thread-4, started daemon 140231621736192)>

DEBUG:root:--- request header ---
DEBUG:root:GET /ob/api/ws/MKS43148972/venues/CEX/tickertape/stocks/AVYO HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: api.stockfighter.io:443
Origin: http://api.stockfighter.io:443
Sec-WebSocket-Key: iGX4AjaiwKtUff+brVUTJA==
Sec-WebSocket-Version: 13


DEBUG:root:--- request header ---
DEBUG:root:GET /ob/api/ws/MKS43148972/venues/CEX/tickertape/stocks/AVYO HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: api.stockfighter.io:443
Origin: http://api.stockfighter.io:443
Sec-WebSocket-Key: iGX4AjaiwKtUff+brVUTJA==
Sec-WebSocket-Version: 13


DEBUG:root:-----------------------
DEBUG:root:--- response header ---
DEBUG:root:-----------------------
DEBUG:root:--- response header ---
DEBUG:root:HTTP/1.1 101 Switching Protocols
DEBUG:root:HTTP/1.1 101 Switching Protocols
DEBUG:root:Server: nginx/1.8.0
DEBUG:root:Server: nginx/1.8.0
DEBUG:root:Date: Wed, 20 Jan 2016 12:52:15 GMT
DEBUG:root:Date: Wed, 20 Jan 2016 12:52:15 GMT
DEBUG:root:Connection: upg